In [100]:
pwd

'/content'

In [101]:
ls

corals/  drive/  sample_data/


In [102]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
cd /content/drive/MyDrive/corals.zip

[Errno 20] Not a directory: '/content/drive/MyDrive/corals.zip'
/content


In [104]:
!unzip /content/drive/MyDrive/corals.zip

Archive:  /content/drive/MyDrive/corals.zip
replace corals/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: corals/test.csv         
  inflating: corals/test/bleached/10309615296_7805d8534c_b.jpg  
  inflating: corals/test/bleached/11263772193_b910d0037b_o.jpg  
  inflating: corals/test/bleached/14157532279_3e38313638_o.jpg  
  inflating: corals/test/bleached/14157549490_03cdd7426c_o.jpg  
  inflating: corals/test/bleached/14548646926_661e9ea246_o.jpg  
  inflating: corals/test/bleached/14606573298_b7e4b33c51_o.jpg  
  inflating: corals/test/bleached/14633767870_660d182ff5_o.jpg  
  inflating: corals/test/bleached/14633816529_af934e0aa8_o.jpg  
  inflating: corals/test/bleached/14789911019_1ec52ae807_b.jpg  
  inflating: corals/test/bleached/14789930360_cc7d42402c_b.jpg  
  inflating: corals/test/bleached/14789940250_d750cd4a53_b.jpg  
  inflating: corals/test/bleached/14817403701_d6453476ed_o.jpg  
  inflating: corals/test/bleached/14818133414_41ea12389a_o.jpg  
  inflat

In [105]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image  import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import softmax
from keras.api._v2.keras import activations
import pandas as pd
from tensorflow.keras import layers, models


In [106]:
# Load the CSV files containing image labels
train_labels = pd.read_csv('/content/corals/train.csv')
test_labels = pd.read_csv('/content/corals/test.csv')

# Display the total number of images and classes
total_train_images = len(train_labels)
total_test_images = len(test_labels)
num_classes = len(train_labels['label'].unique())

print(f'Total training images: {total_train_images}')
print(f'Total testing images: {total_test_images}')
print(f'Number of classes: {num_classes}')
batch_size=32

Total training images: 738
Total testing images: 185
Number of classes: 2


In [113]:
train_datagen = ImageDataGenerator(
rescale=1./255,
zoom_range=0.2,
horizontal_flip=True,
validation_split=0.2)

In [115]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)

In [116]:
#x_train = train_datagen.flow_from_directory()
train_set = train_datagen.flow_from_directory('/content/corals/train',
                                              target_size=(299,299),
                                              batch_size=32,
                                              class_mode= 'categorical',
                                              subset='training')

Found 591 images belonging to 2 classes.


In [117]:
train_set.class_indices

{'bleached': 0, 'healthy': 1}

In [118]:
test_set = test_datagen.flow_from_directory('/content/corals/test',
                                            target_size=(299,299),
                                            batch_size=32,
                                            class_mode='categorical',
                                            subset='validation')

Found 36 images belonging to 2 classes.


#**VGG16**

In [119]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D,Flatten,Dense

In [120]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [121]:
vgg = VGG16(weights='imagenet',include_top = False,input_shape = (299,299,3))

In [122]:
for layer in vgg.layers:
  print(layer)

In [123]:
len(vgg.layers)

19

In [124]:
# Freeze the layers of the pre-trained model
for layer in vgg.layers:
    layer.trainable = False

# Create a new model using the VGG16 base
vggmodel = models.Sequential()
vggmodel.add(vgg)

# Flatten the output and add dense layers for classification
vggmodel.add(layers.Flatten())
vggmodel.add(layers.Dense(256, activation='relu'))
vggmodel.add(layers.Dropout(0.5))
vggmodel.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
vggmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
vggmodel.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 9, 9, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 41472)             0         
                                                                 
 dense_8 (Dense)             (None, 256)               10617088  
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 2)                 514       
                                                                 
Total params: 25332290 (96.64 MB)
Trainable params: 10617602 (40.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [125]:
epochs = 10

history = vggmodel.fit(
    train_set,
    steps_per_epoch=train_set.samples // batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size
)


Epoch 1/10
18/18 [==============================] - 18s 917ms/step - loss: 1.8706 - accuracy: 0.6011 - val_loss: 0.7518 - val_accuracy: 0.6250
Epoch 2/10
18/18 [==============================] - 16s 889ms/step - loss: 0.6166 - accuracy: 0.6959 - val_loss: 0.5314 - val_accuracy: 0.6875
Epoch 3/10
18/18 [==============================] - 17s 938ms/step - loss: 0.5416 - accuracy: 0.7263 - val_loss: 0.5971 - val_accuracy: 0.6562
Epoch 4/10
18/18 [==============================] - 17s 920ms/step - loss: 0.5185 - accuracy: 0.7174 - val_loss: 0.5945 - val_accuracy: 0.7500
Epoch 5/10
18/18 [==============================] - 16s 875ms/step - loss: 0.4852 - accuracy: 0.7603 - val_loss: 0.5410 - val_accuracy: 0.8438
Epoch 6/10
18/18 [==============================] - 17s 935ms/step - loss: 0.4639 - accuracy: 0.7746 - val_loss: 0.5626 - val_accuracy: 0.7812
Epoch 7/10
18/18 [==============================] - 17s 928ms/step - loss: 0.4649 - accuracy: 0.7835 - val_loss: 0.6419 - val_accuracy: 0.7812

In [126]:
from keras.optimizers import Adam
#opt = Adam(lr=0.001)

vggmodel.compile(loss = 'categorical_crossentropy' , metrics =['accuracy'])

In [21]:
labels = ['bleached','healthy']

In [22]:
import numpy as np
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [237]:
import numpy as np
img = '/content/corals/test/healthy/14856466066_4c8af3c82e_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds = vggmodel.predict(np.array([x]))
preds

1/1 [==============================] - 0s 30ms/step


array([[6.0504203e-04, 9.9999464e-01]], dtype=float32)

In [238]:
labels[np.argmax(preds)]

'healthy'

In [217]:
import numpy as np
img = '/content/corals/test/bleached/22320951345_32a7d81f21_b.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds1 = vggmodel.predict(np.array([x]))
preds1

1/1 [==============================] - 0s 26ms/step


array([[1.000000e+00, 9.092037e-08]], dtype=float32)

In [218]:
labels[np.argmax(preds1)]

'bleached'

#**Resnet**


In [127]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models, layers

# Load the pre-trained ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of the pre-trained model
for layer in resnet.layers:
    layer.trainable = False

# Create a new model using the ResNet50 base
resnet_model = models.Sequential()
resnet_model.add(resnet)

# Flatten the output and add dense layers for classification
resnet_model.add(layers.Flatten())
resnet_model.add(layers.Dense(256, activation='relu'))
resnet_model.add(layers.Dropout(0.5))
resnet_model.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
resnet_model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 10, 10, 2048)      23587712  
                                                                 
 flatten_3 (Flatten)         (None, 204800)            0         
                                                                 
 dense_10 (Dense)            (None, 256)               52429056  
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_11 (Dense)            (None, 2)                 514       
                                                                 
Total params: 76017282 (289.98 MB)
Trainable params: 52429570 (200.00 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [128]:
epochs = 10
batch_size = 32  # Adjust the batch size based on your requirements

history= resnet_model.fit(
    train_set,
    steps_per_epoch=train_set.samples // batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size
)


Epoch 1/10
18/18 [==============================] - 25s 950ms/step - loss: 14.7002 - accuracy: 0.5063 - val_loss: 2.5426 - val_accuracy: 0.4375
Epoch 2/10
18/18 [==============================] - 15s 812ms/step - loss: 1.1914 - accuracy: 0.5224 - val_loss: 0.6932 - val_accuracy: 0.4375
Epoch 3/10
18/18 [==============================] - 15s 848ms/step - loss: 0.6931 - accuracy: 0.5116 - val_loss: 0.6931 - val_accuracy: 0.5312
Epoch 4/10
18/18 [==============================] - 15s 829ms/step - loss: 0.7609 - accuracy: 0.5098 - val_loss: 0.6930 - val_accuracy: 0.5625
Epoch 5/10
18/18 [==============================] - 15s 850ms/step - loss: 0.6931 - accuracy: 0.5170 - val_loss: 0.6930 - val_accuracy: 0.5938
Epoch 6/10
18/18 [==============================] - 15s 798ms/step - loss: 0.6931 - accuracy: 0.5134 - val_loss: 0.6929 - val_accuracy: 0.5938
Epoch 7/10
18/18 [==============================] - 15s 786ms/step - loss: 0.7659 - accuracy: 0.5206 - val_loss: 0.6929 - val_accuracy: 0.562

In [36]:
from keras.optimizers import Adam
#opt = Adam(lr=0.001)

resnet_model.compile(loss = 'categorical_crossentropy' , metrics =['accuracy'])

In [37]:
labels = ['bleached','healthy']

In [255]:
img = '/content/corals/test/healthy/12678987553_66ddc5afd9_b.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds = resnet_model.predict(np.array([x])).flatten()
preds

1/1 [==============================] - 0s 115ms/step


array([0.23467752, 0.47156852], dtype=float32)

In [256]:
labels[np.argmax(preds)]

'healthy'

In [257]:
img = '/content/corals/test/bleached/14633767870_660d182ff5_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds1= resnet_model.predict(np.array([x])).flatten()
preds1

1/1 [==============================] - 0s 53ms/step


array([0.43522742, 0.16685407], dtype=float32)

In [258]:
labels[np.argmax(preds1)]

'bleached'

#**Inception**

In [129]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import models, layers

# Load the pre-trained InceptionV3 model
inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of the pre-trained model
for layer in inception.layers:
    layer.trainable = False

# Create a new model using the InceptionV3 base
inception_model = models.Sequential()
inception_model.add(inception)

# Add a Global Average Pooling layer
inception_model.add(layers.GlobalAveragePooling2D())

# Add dense layers for classification
inception_model.add(layers.Dense(256, activation='relu'))
inception_model.add(layers.Dropout(0.5))
inception_model.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
inception_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
inception_model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_12 (Dense)            (None, 256)               524544    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 2)                 514       
                                                                 
Total params: 22327842 (85.17 MB)
Trainable params: 525058 (2.00 MB)
Non-trainable params: 21802784 (83.17 MB)
_________

In [130]:
epochs = 10
batch_size = 32  # Adjust the batch size based on your requirements

history= inception_model.fit(
    train_set,
    steps_per_epoch=train_set.samples // batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size
)

Epoch 1/10
18/18 [==============================] - 21s 885ms/step - loss: 0.6863 - accuracy: 0.6225 - val_loss: 0.5913 - val_accuracy: 0.6562
Epoch 2/10
18/18 [==============================] - 14s 793ms/step - loss: 0.5863 - accuracy: 0.6869 - val_loss: 0.5707 - val_accuracy: 0.7188
Epoch 3/10
18/18 [==============================] - 14s 802ms/step - loss: 0.5474 - accuracy: 0.7156 - val_loss: 0.5487 - val_accuracy: 0.7188
Epoch 4/10
18/18 [==============================] - 14s 756ms/step - loss: 0.5198 - accuracy: 0.7567 - val_loss: 0.4796 - val_accuracy: 0.7812
Epoch 5/10
18/18 [==============================] - 14s 797ms/step - loss: 0.5056 - accuracy: 0.7335 - val_loss: 0.5586 - val_accuracy: 0.6875
Epoch 6/10
18/18 [==============================] - 14s 797ms/step - loss: 0.5073 - accuracy: 0.7406 - val_loss: 0.5793 - val_accuracy: 0.6250
Epoch 7/10
18/18 [==============================] - 14s 798ms/step - loss: 0.4956 - accuracy: 0.7478 - val_loss: 0.5338 - val_accuracy: 0.6562

In [44]:
from keras.optimizers import Adam
#opt = Adam(lr=0.001)

inception_model.compile(loss = 'categorical_crossentropy' , metrics =['accuracy'])

In [45]:
labels = ['bleached','healthy']

In [46]:
img = '//content/corals/test/healthy/37391917484_2974d97d44_b.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds = inception_model.predict(np.array([x])).flatten()
preds

1/1 [==============================] - 3s 3s/step


array([2.8672073e-12, 1.0000000e+00], dtype=float32)

In [47]:
labels[np.argmax(preds)]

'healthy'

In [269]:
img = '/content/corals/test/bleached/14157549490_03cdd7426c_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds1 = inception_model.predict(np.array([x])).flatten()
preds1

1/1 [==============================] - 0s 26ms/step


array([0.9990319, 0.0054231], dtype=float32)

In [270]:
labels[np.argmax(preds1)]

'bleached'

#**Densenet**


In [131]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import models, layers

# Load the pre-trained DenseNet121 model
densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of the pre-trained model
for layer in densenet.layers:
    layer.trainable = False

# Create a new model using the DenseNet121 base
densenet_model = models.Sequential()
densenet_model.add(densenet)

# Add a Global Average Pooling layer
densenet_model.add(layers.GlobalAveragePooling2D())

# Add dense layers for classification
densenet_model.add(layers.Dense(256, activation='relu'))
densenet_model.add(layers.Dropout(0.5))
densenet_model.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
densenet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
densenet_model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 9, 9, 1024)        7037504   
                                                                 
 global_average_pooling2d_3  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_14 (Dense)            (None, 256)               262400    
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                                 
 dense_15 (Dense)            (None, 2)                 514       
                                                                 
Total params: 7300418 (27.85 MB)
Trainable params: 262914 (1.00 MB)
Non-trainable params: 7037504 (26.85 MB)
___________

In [132]:
epochs = 10
batch_size = 32  # Adjust the batch size based on your requirements

history= densenet_model.fit(
    train_set,
    steps_per_epoch=train_set.samples // batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size
)

Epoch 1/10
18/18 [==============================] - 24s 972ms/step - loss: 0.6851 - accuracy: 0.6369 - val_loss: 0.6017 - val_accuracy: 0.6875
Epoch 2/10
18/18 [==============================] - 15s 818ms/step - loss: 0.5751 - accuracy: 0.6995 - val_loss: 0.6435 - val_accuracy: 0.6562
Epoch 3/10
18/18 [==============================] - 14s 769ms/step - loss: 0.4909 - accuracy: 0.7531 - val_loss: 0.5803 - val_accuracy: 0.6875
Epoch 4/10
18/18 [==============================] - 15s 803ms/step - loss: 0.4858 - accuracy: 0.7710 - val_loss: 0.4287 - val_accuracy: 0.8438
Epoch 5/10
18/18 [==============================] - 15s 793ms/step - loss: 0.4569 - accuracy: 0.7674 - val_loss: 0.5348 - val_accuracy: 0.8125
Epoch 6/10
18/18 [==============================] - 15s 810ms/step - loss: 0.4991 - accuracy: 0.7549 - val_loss: 0.5214 - val_accuracy: 0.7500
Epoch 7/10
18/18 [==============================] - 14s 807ms/step - loss: 0.4210 - accuracy: 0.8157 - val_loss: 0.5745 - val_accuracy: 0.6875

In [279]:
img = '/content/corals/test/healthy/10167343533_ea141f7e70_b.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds = densenet_model.predict(np.array([x])).flatten()
preds

1/1 [==============================] - 0s 52ms/step


array([0.5713068, 0.8180201], dtype=float32)

In [280]:
labels[np.argmax(preds)]

'healthy'

In [277]:
img = '/content/corals/test/bleached/11263772193_b910d0037b_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds1 = densenet_model.predict(np.array([x])).flatten()
preds1

1/1 [==============================] - 0s 56ms/step


array([0.993729 , 0.0273751], dtype=float32)

In [278]:
labels[np.argmax(preds1)]

'bleached'

#**Xpection**

In [133]:
from tensorflow.keras.applications import Xception
from tensorflow.keras import models, layers

# Load the pre-trained Xception model
xception = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of the pre-trained model
for layer in xception.layers:
    layer.trainable = False

# Create a new model using the Xception base
xception_model = models.Sequential()
xception_model.add(xception)

# Add a Global Average Pooling layer
xception_model.add(layers.GlobalAveragePooling2D())

# Add dense layers for classification
xception_model.add(layers.Dense(256, activation='relu'))
xception_model.add(layers.Dropout(0.5))
xception_model.add(layers.Dense(2, activation='sigmoid'))

# Compile the model
xception_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
xception_model.summary()


83683744/83683744 [==============================] - 0s 0us/step
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d_4  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_16 (Dense)            (None, 256)               524544    
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 dense_17 (Dense)            (None, 2)                 514       
                                                                 
Total params: 21386538 (81.58 MB)
Trainable params: 525

In [134]:
epochs = 10
batch_size = 32  # Adjust the batch size based on your requirements

history= xception_model.fit(
    train_set,
    steps_per_epoch=train_set.samples // batch_size,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size
)

Epoch 1/10
18/18 [==============================] - 26s 974ms/step - loss: 0.6225 - accuracy: 0.6476 - val_loss: 0.6785 - val_accuracy: 0.6250
Epoch 2/10
18/18 [==============================] - 15s 830ms/step - loss: 0.5519 - accuracy: 0.7102 - val_loss: 0.5940 - val_accuracy: 0.5938
Epoch 3/10
18/18 [==============================] - 15s 828ms/step - loss: 0.5401 - accuracy: 0.7496 - val_loss: 0.6180 - val_accuracy: 0.6250
Epoch 4/10
18/18 [==============================] - 15s 829ms/step - loss: 0.4765 - accuracy: 0.7835 - val_loss: 0.5353 - val_accuracy: 0.6562
Epoch 5/10
18/18 [==============================] - 15s 833ms/step - loss: 0.4905 - accuracy: 0.7603 - val_loss: 0.5671 - val_accuracy: 0.6562
Epoch 6/10
18/18 [==============================] - 15s 854ms/step - loss: 0.4530 - accuracy: 0.7835 - val_loss: 0.5348 - val_accuracy: 0.6562
Epoch 7/10
18/18 [==============================] - 15s 831ms/step - loss: 0.4299 - accuracy: 0.8014 - val_loss: 0.6249 - val_accuracy: 0.6250

In [283]:
img = '//content/corals/test/bleached/14157549490_03cdd7426c_o.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds = xception_model.predict(np.array([x])).flatten()
preds

1/1 [==============================] - 0s 48ms/step


array([0.3957039 , 0.17180482], dtype=float32)

In [284]:
labels[np.argmax(preds)]

'bleached'

In [281]:
img = '/content/corals/test/healthy/13973931311_daf38429eb_b.jpg'
img = load_img(img, target_size=(299,299))
x = img_to_array(img)
x = preprocess_input(x)
preds1 = xception_model.predict(np.array([x])).flatten()
preds1

1/1 [==============================] - 1s 1s/step


array([1.2375622e-09, 1.0000000e+00], dtype=float32)

In [282]:
labels[np.argmax(preds1)]

'healthy'

In [285]:
#model.save('space_img.h5')
vggmodel.save('waves_img.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
